In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import csv

%matplotlib inline

In [ ]:
# display all pandas columns
pd.set_option('display.max_columns', None)

### Open and Examine Data

In [ ]:
input_csv_path = './input_ai_data.csv'

In [ ]:
def load_input_csv_to_dataframe(input_csv_path: str) -> pd.DataFrame:
    # load all lines into list
    all_lines = []

    with open(input_csv_path, "r") as f:
        reader = csv.reader(f, delimiter=",")
        for line in tqdm(reader):
            all_lines.append(line[:31])

    # make into dataframe
    df = pd.DataFrame(all_lines)
    df.columns = df.iloc[0]
    df = df[1:]
    
    return df

df = load_input_csv_to_dataframe(input_csv_path)

In [ ]:
# get columns relevent to remark rating
df = df[['Category ID', 'Player Level', 'User Note', 'Occurrence Remarks', 'Expert Remark Rating', 'Expert Level', 'Expert Field Note Rating']]
df = df.drop_duplicates()
print(df.shape)

In [ ]:
# take all before taxon tree (what was added post processing)
df['User Note'] = df['User Note'].apply(lambda x: x.split('Taxon Tree')[0])

In [ ]:
df.head(2)

In [ ]:
df.sample(1)['Occurrence Remarks'].iloc[0]

In [ ]:
# make sure all occurrence remarks are valid
df = df[df['Occurrence Remarks'] != '']
print(df.shape)

In [ ]:
# drop rows where nan expert remark rating
df = df[df['Expert Remark Rating'] != '']

In [ ]:
sns.countplot(x='Expert Remark Rating', data=df)
plt.title('Expert Remark Distribution');

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
# turn expert remark rating into an int
df['Expert Remark Rating'] = df['Expert Remark Rating'].apply(lambda x: int(float(x)))

In [ ]:
df.shape

### Preprocess text
https://machinelearningknowledge.ai/11-techniques-of-text-preprocessing-using-nltk-in-python/

In [ ]:
df.head(1)

In [ ]:
#!pip install nltk 

In [ ]:
import re
import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
en_stopwords = stopwords.words('english')


def remove_whitespace(text):
    return  " ".join(text.split())


def remove_stopwords(text):
    result = [token for token in text if token not in en_stopwords]
    return result


def remove_punct(text):
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst


def lemmatization(text):
    result=[]
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
            
        result.append(wordnet.lemmatize(token,pos))
    
    return result


def remove_tag(text):
    text=' '.join(text)
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)


def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [ ]:
subset_df = df#sample(10_000).copy() # take subset of 10000 first

In [ ]:
tqdm.pandas()
print('Removing Whitespace')
subset_df['Occurrence Remarks'] = subset_df['Occurrence Remarks'].progress_apply(remove_whitespace)

print('Tokenizing words')
subset_df['Occurrence Remarks'] = subset_df['Occurrence Remarks'].progress_apply(word_tokenize) 

print('Removing stopwords')
subset_df['Occurrence Remarks'] = subset_df['Occurrence Remarks'].progress_apply(lambda x: remove_stopwords(x)) 

print('Removing punctuation')
subset_df['Occurrence Remarks'] = subset_df['Occurrence Remarks'].progress_apply(lambda x: " ".join(remove_punct(x)))

print('Removing URLs')
subset_df['Occurrence Remarks'] = subset_df['Occurrence Remarks'].progress_apply(remove_urls) 

print('Lemmatizing')
subset_df['Occurrence Remarks'] = subset_df['Occurrence Remarks'].progress_apply(lambda x: " ".join(lemmatization(x.split())))

In [ ]:
# include player encoding into string; of category and level, see if it makes a difference.
df['Occurrence Remarks'] = df.apply(lambda x: f"player level {x['Player Level']}. category level {x['Category ID']}. {x['Occurrence Remarks']}", axis=1)

In [ ]:
# create and save text file
with open('./remarks.txt', 'w+') as wf:
    for el, jun in subset_df[['Expert Remark Rating', 'Occurrence Remarks']].values:
        string = f"__label__{el} {jun}"
        wf.write(string)
        wf.write('\n')

In [ ]:
# for total of 10000 data points
!head -n 300000 remarks.txt > remarks.train
!tail -n 11918 remarks.txt > remarks.valid

### Train Using FastText
https://fasttext.cc/docs/en/supervised-tutorial.html

In [ ]:
import fasttext

In [ ]:
model = fasttext.train_supervised(input="remarks.train", lr=1, epoch=100, wordNgrams=3)

In [ ]:
#model.save_model("model_remarks.bin") #load_model does the inverse

# model.test("remarks.valid") # precision, recall at 1; how often highest ranked document contains correct answer
model.test("remarks.valid", k=5) # precision, recall at 5. want > 0.2 precision, otherwise no better than random

In [ ]:
model.predict("player level 10. category level 23. what an amazing kangaroo", k=-1)

In [ ]:
model.test("remarks.valid", k=-1)

### Results are not that great; let's try BERT
-  MultiLabel Classification: https://colab.research.google.com/github/rap12391/transformers_multilabel_toxic/blob/master/toxic_multilabel.ipynb#scrollTo=0DF3ddjej5vd
-  MultiClass Classification: https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb

In [ ]:
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset

In [ ]:
num_labels = len(subset_df['Expert Remark Rating'].unique())

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(subset_df.index.values, 
                                                  subset_df['Expert Remark Rating'].values, 
                                                  test_size=0.10, 
                                                  random_state=42, 
                                                  stratify=subset_df['Expert Remark Rating'].values)

In [ ]:
subset_df['data_type'] = ['not_set']*subset_df.shape[0]

subset_df.loc[X_train, 'data_type'] = 'train'
subset_df.loc[X_val, 'data_type'] = 'val'

In [ ]:
subset_df.groupby(['Expert Remark Rating', 'data_type']).count()

In [ ]:
# load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    subset_df[subset_df.data_type=='train']['Occurrence Remarks'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=256, # tested 512 here, doesn't make much of a difference
    return_tensors='pt',
    truncation=True
)

encoded_data_val = tokenizer.batch_encode_plus(
    subset_df[subset_df.data_type=='val']['Occurrence Remarks'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=256, # can go up to 512 but we run into some memory issues..
    return_tensors='pt',
    truncation=True
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(subset_df[subset_df.data_type=='train']['Expert Remark Rating'].values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(subset_df[subset_df.data_type=='val']['Expert Remark Rating'].values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=num_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16 

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
subset_df['Expert Remark Rating'].unique() # can try to predict results as-is

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

# see what gpus are available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'models/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    
HBox(children=(FloatProgress(value=0.0, max=5.0), HTML(value='')))
HBox(children=(FloatProgress(value=0.0, description='Epoch ')))

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=num_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
model.load_state_dict(torch.load('./models/finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)
flattened_predictions = np.argmax(predictions, axis=1).flatten()

In [ ]:
# !pip install sklearn
import sklearn
from sklearn.metrics import classification_report
print(sklearn.metrics.classification_report(true_vals, flattened_predictions))

In [ ]:
accuracy_per_class(predictions, true_vals)

### Load and Make Predictions on Sample String

In [ ]:
import torch
import numpy as np

from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from transformers import BertForSequenceClassification


def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
import re
import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
en_stopwords = stopwords.words('english')

def remove_whitespace(text):
    return  " ".join(text.split())


def remove_stopwords(text):
    result = [token for token in text if token not in en_stopwords]
    return result


def remove_punct(text):
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst


def lemmatization(text):
    result=[]
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
            
        result.append(wordnet.lemmatize(token,pos))
    
    return result


def remove_tag(text):
    text=' '.join(text)
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)


def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)


def preprocess_input_text(text):
    text = remove_whitespace(text)
    text = word_tokenize(text)
    text = remove_stopwords(text)
    text = remove_punct(text)
    text = lemmatization(text)
    text = " ".join(text)
    text = remove_urls(text)
    return text


In [ ]:
SAMPLE_STRING = """
A good sighting of the Superb Fairywren (Malurus cyaneus). 

-------------
Adult male Superb Fairy-wrens are among the most brightly coloured of the species, especially during the breeding season. They have rich blue and black plumage above and on the throat. The belly is grey-white and the bill is black. Females and young birds are mostly brown above with a dull red-orange area around the eye and a brown bill. Females have a pale greenish gloss, absent in young birds, on the otherwise brown tail. The legs are brown in both sexes. Males from further inland and in the south-west of the range have more blue on the back and underparts.

Similar species: 
Several other species of fairy-wren are found in Australia. The males of each species are quite distinct, but the females and young birds are often difficult to separate. Of the species that overlap in range with the Superb Fairy-wren, the female White-winged Fairy-wren Malurus leucopterus and Red-backed Fairy-wren M. melanocephalus lacks the chestnut colour around the eye, while the female Variegated Fairy-wren M. lamberti has a dull grey-blue wash. Both the Superb and White-winged Fairy-wrens are similar in size. The Variegated Fairy-wren is slightly larger in size and has a longer tail.

Where does it live?
Distribution: 
Superb Fairy-wrens are found south of the Tropic of Capricorn through eastern Australia and Tasmania to the south-eastern corner of South Australia.

Habitat: 
Seen in most habitat types where suitable dense cover and low shrubs occur. They are common in urban parks and gardens, and can be seen in small social groups. These groups normally consist of one dominant male and several females and young birds.

What does it do?
Feeding: 
Superb Fairy-wrens feed on insects and other small arthropods. These are caught mostly on the ground, but may also be taken from low bushes. Feeding takes place in small social groups.

Breeding: 
The nest is a dome-shaped structure of grasses and other fine material. It is usually placed in a low bush and is constructed by the female. The female incubates the eggs alone, but both sexes feed the young. Other members of the group will also help with the feeding of the young.

References: 
Field Guide to the Birds of Australia.
Handbook of Australian, New Zealand and Antarctic Birds, Volume 5 {(Tyrant-flycatchers} to Chats).
Reader's Digest Complete Book of Australian Birds
The Wrens and Warblers of Australia
"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_labels = 6

# load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=num_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.to(device)
model.load_state_dict(torch.load('./models/finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

# preprocess
processed_sample_string = [preprocess_input_text(SAMPLE_STRING)]

# tokenize
encoded_string = tokenizer.batch_encode_plus(
    processed_sample_string, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=512, # can go up to 512 but we run into some memory issues..
    return_tensors='pt',
    truncation=True
)

# separate to input ids, attention mask and labels
input_ids_sample = encoded_string['input_ids']
attention_masks_sample = encoded_string['attention_mask']
labels_sample = torch.tensor(np.ones(len(processed_sample_string))).type(torch.LongTensor)

# create tensor dataset
dataset_sample = TensorDataset(input_ids_sample, attention_masks_sample, labels_sample)
dataloader_sample = DataLoader(dataset_sample, sampler=SequentialSampler(dataset_sample), batch_size=1)

In [ ]:
_, predictions, true_vals = evaluate(dataloader_sample)

In [ ]:
predictions.argmax()